In [1]:
# 把层当做函数来使用,接受张量并返回张量
from keras import layers, Input
# 一个张量
input_tensor = Input(shape=(32, ))
# 一个层是一个函数
dense = layers.Dense(32, activation='relu')
# 可以在一个张量上调用一个层,它会返回另一个张量
output_tensor = dense(input_tensor)


Using TensorFlow backend.


# 并列展示一个简单的Sequential模型以及对应的函数式API实现

In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# Sequential模型
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64, )))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

print(seq_model.summary())

# 对应的函数式API实现
input_tensor = Input(shape=(64, ))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)
# Model类将输入张量和输出张量转换为一个模型
model = Model(input_tensor, output_tensor)
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
den

In [3]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))
model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 1s 533us/step - loss: 11.5583
Epoch 2/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.5207
Epoch 3/10
1000/1000 [==============================] - 0s 14us/step - loss: 11.5106
Epoch 4/10
1000/1000 [==============================] - 0s 15us/step - loss: 11.5055
Epoch 5/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.5012
Epoch 6/10
1000/1000 [==============================] - 0s 11us/step - loss: 11.4983
Epoch 7/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.4958
Epoch 8/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.4949
Epoch 9/10
1000/1000 [==============================] - 0s 13us/step - loss: 11.4917
Epoch 10/10
1000/1000 [==============================] - 0s 29us/step


# 多输入模型

## 用函数式API实现双输入问答模型

In [5]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# 文本输入是一个长度可变的整数序列
text_input = Input(shape=(None, ), dtype=np.int32, name='text')
# 将输入嵌入长度为64的向量
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
# 利用LSTM将向量编码为单个向量
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None, ), dtype=np.int32, name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(32)(embedded_question)

# 将编码后的问题和文本连接起来
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax', name='anwer')(concatenated)

# 在模型实例化时,指定两个输入和一个输出
model = Model([text_input, question_input], answer)

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['acc']
)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_3 (LS

## 将数据输入到多输入模型中

In [7]:
import numpy as np
import keras

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = keras.utils.to_categorical(answers, answer_vocabulary_size)

# model.fit([text, question], answers, epochs=10, batch_size=128)
model.fit({'text':text, 'question':question}, {'anwer':answers}, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.2518 - acc: 0.0040
Epoch 2/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1982 - acc: 0.0050
Epoch 3/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.1562 - acc: 0.0060
Epoch 4/10
1000/1000 [==============================] - 1s 1ms/step - loss: 6.0690 - acc: 0.0080
Epoch 5/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.9723 - acc: 0.0170
Epoch 6/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.8564 - acc: 0.0200
Epoch 7/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.7855 - acc: 0.0240
Epoch 8/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.6770 - acc: 0.0300
Epoch 9/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.5861 - acc: 0.0320
Epoch 10/10
1000/1000 [==============================] - 1s 1ms/step - loss: 5.5032 - acc: 0.0320


# 多输出模型

## 用函数式API实现一个三输出模型

In [9]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None, ), dtype=np.int32, name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(10, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 128)    32000128    embedding_5[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 128)    0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (

## 多输出模型的编译选项:多重损失

In [11]:
# model.compile(
#     optimizer='rmsprop',
#     loss=['mse', 'categorical_crossentropy', 'binary_crossentropy']
# )

# 只有输出层具有名称时,才能采用这种写法
model.compile(
    optimizer='rmsprop',
    loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'}
)

## 多输出模型的编译选项:损失加权

In [14]:
model.compile(
    optimizer='rmsprop',
    loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
    loss_weights=[0.25, 1.0, 10.0]
)

# 只有输出层具有名称时,才能采用这种写法
model.compile(
    optimizer='rmsprop',
    loss={'age': 'mse', 'income': 'categorical_crossentropy', 'gender': 'binary_crossentropy'},
    loss_weights={'age': 0.25, 'income': 1.0, 'gender': 10.0}
)

## 将数据输入到多输出模型中

In [ ]:
model.fit(
    posts,
    [age_targets, income_targets, gender_targets],
    epochs=10,
    batch_size=64
)

# 只有输出层具有名称时,才能采用这种写法
model.fit(
    posts,
    {'age': age_targets, 'income': income_targets, 'gender': gender_targets},
    epochs=10,
    batch_size=64
)

# 层组成的有向无环图DAG

## Inception模块

In [23]:
from keras import layers, Input
from keras.models import Model

x = Input(shape=(300, 300, 3), dtype=np.float32, name='input')

branch_a = layers.Conv2D(128, 1, strides=2, activation='relu')(x)

branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, strides=2, activation='relu')(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, strides=2, activation='relu')(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)



In [21]:
from keras.applications.inception_v3 import InceptionV3
net = InceptionV3(weights='imagenet',
                 include_top=False,
                 input_shape=(300, 300, 3))

87916544/87910968 [==============================] - 18s 0us/step


In [22]:
net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_105 (Conv2D)             (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_105[0][0]                 
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 149, 149, 32) 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_106

## 残差连接

In [31]:
# 如果特征图的尺寸相同
from keras import layers, Input
x = Input(shape=(300, 300, 3), dtype=np.float32, name='input')
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
# 将原始x与输出特征相加
y = layers.add([y, x])


In [32]:
# 如果特征图的尺寸不同
from keras import layers, Input
x = Input(shape=(300, 300, 3), dtype=np.float32, name='input')
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)
# 使用1x1卷积,将原始x张量线性下采样为与y具有相同的形状
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)
# 将残差张量与输出特征相加
y = layers.add([y, residual])
net = Model(x, y)
net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_230 (Conv2D)             (None, 300, 300, 128 3584        input[0][0]                      
__________________________________________________________________________________________________
conv2d_231 (Conv2D)             (None, 300, 300, 128 147584      conv2d_230[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_13 (MaxPooling2D) (None, 150, 150, 128 0           conv2d_231[0][0]                 
__________________________________________________________________________________________________
conv2d_232

# 共享层权重

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)

left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)
model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)

# 将模型作为层

In [ ]:
from keras import layers
from keras import applications
from keras import Input

xception_base = applications.Xception(weight=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_featrures = xception_base(left_input)
right_input = xception_base(right_input)

merged_features = layers.concatenate([left_featrures, right_input], axis=-1)